Load data, clean the data, combine separte data together, divide training data and test data.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#outdoor air temperature
oa = pd.read_csv("../data/oa_temp_utc_f.csv");
oa.columns = ['time', 'oa']
oa.set_index("time", drop = True, inplace = True);

oa.index = pd.to_datetime(oa.index)
oa = oa.replace('/', np.nan)
oa['oa'] = oa['oa'].astype(float)
oa = oa.interpolate(method = "time")

#relative humidity
rh = pd.read_csv("../data/rh_utc_perct.csv");
rh.columns = ['time', 'rh']
rh.set_index("time", drop = True, inplace = True);

rh.index = pd.to_datetime(rh.index)
rh = rh.replace('/', np.nan)
rh['rh'] = rh['rh'].astype(float)
rh = rh.interpolate(method = "time")

#solar radiation
sr = pd.read_csv("../data/total_solar_utc_btuh-ft2.csv");
sr.columns = ['time', 'sr'];
sr.set_index('time', drop = True, inplace = True);

sr.index = pd.to_datetime(sr.index)
sr = sr.replace('/', np.nan)
sr['sr'] = sr['sr'].astype(float)
sr = sr.interpolate(method = "time")
#wind speed
ws = pd.read_csv("../data/wind_speed_utc_mph.csv");
ws.columns = ['time', 'ws'];
ws.set_index('time', drop = True, inplace = True);

ws.index = pd.to_datetime(ws.index)
ws = ws.replace('/', np.nan)
ws['ws'] = ws['ws'].astype(float)
ws = ws.interpolate(method = "time")
#damper position of room 107n
dp = pd.read_csv("../data/107n_damper_utc_0to7or10.csv");
dp.columns = ['time', 'dp'];
dp.set_index('time', drop = True, inplace = True); 

dp.index = pd.to_datetime(dp.index)
dp = dp.replace('/', np.nan)
dp['dp'] = dp['dp'].astype(float)
dp = dp.interpolate(method = "time")
#some data has scale 0-10, some has scale 0-7, so transfer 0-7 to 0-10
dpUpper = dp.loc['2014-06-26 00:10:00':'2015-02-02 14:40:00',:];
dpLower = dp.loc['2015-02-02 14:50:00':,:]
dpLower = dpLower.multiply(10.0/7.0, axis='columns');

frames = [dpUpper, dpLower]
dp = pd.concat(frames)

#supply air temperature of room 107n
st = pd.read_csv("../data/107n_vavtemp_utc_f.csv");
st.columns = ['time', 'st'];
st.set_index('time', drop = True, inplace = True);

st.index = pd.to_datetime(st.index)
st = st.replace('/', np.nan)
st['st'] = st['st'].astype(float)
st = st.interpolate(method = "time");

#indoor air temperature of 107n
at = pd.read_csv("../data/107n_temp_utc_f.csv");
at.columns = ['time', 'at'];
at.set_index('time', drop = True, inplace = True);

at.index = pd.to_datetime(at.index)
at = at.replace('/', np.nan)
at['at'] = at['at'].astype(float)
at = at.interpolate(method = "time");

#merge together, change original utc time to local time
allDataRaw = (oa.merge(rh,how = 'inner',left_index = True, right_index = True)
              .merge(sr,how = 'inner',left_index = True, right_index = True)
              .merge(ws,how = 'inner',left_index = True, right_index = True)
              .merge(dp,how = 'inner',left_index = True, right_index = True)
              .merge(st,how = 'inner',left_index = True, right_index = True)
              .merge(at,how = 'inner',left_index = True, right_index = True));
import pytz
eastern = pytz.timezone('US/Eastern');
origIndex = allDataRaw.index;
newTimeIndex = origIndex.tz_localize(pytz.utc).tz_convert(eastern);
allDataRaw.index = newTimeIndex;

#add month, weekday, hour of day information
allDataRaw['month'] = allDataRaw.index;
allDataRaw['weekday'] = allDataRaw.index;
allDataRaw['hour'] = allDataRaw.index;
allDataRaw['month'] = (allDataRaw['month']
                             .apply(lambda x: x.month))
allDataRaw['weekday'] = (allDataRaw['weekday']
                             .apply(lambda x: x.weekday()))
allDataRaw['hour'] = (allDataRaw['hour']
                             .apply(lambda x: x.hour));

#seperate one year for train, one year for test
train = allDataRaw.loc['2014-11-01 00:00:00':'2015-11-01 00:00:00',:];
test = allDataRaw.loc['2015-11-01 00:10:00':'2016-11-01 00:10:00'];
train

,oa,rh,sr,ws,dp,st,at,month,weekday,hour
time,,,,,,,,,,
2014-11-01 00:00:00-04:00,42.700000,86.589980,0.704026,0.000000,0.000000,62.440970,68.200000,11,5,0
2014-11-01 00:10:00-04:00,42.710010,86.600000,1.244994,0.000000,0.000000,62.341324,68.100000,11,5,0
2014-11-01 00:20:00-04:00,42.800000,86.600000,1.115994,0.000000,0.000000,63.057070,68.100000,11,5,0
2014-11-01 00:30:00-04:00,42.900000,86.500000,0.900000,0.062532,0.000000,64.773110,68.100000,11,5,0
2014-11-01 00:40:00-04:00,42.900000,86.410010,0.560000,0.000000,0.000000,66.950250,67.946335,11,5,0
2014-11-01 00:50:00-04:00,42.900000,86.500000,0.584016,0.000000,0.000000,68.192020,67.900000,11,5,0
2014-11-01 01:00:00-04:00,43.000000,86.510025,1.250954,0.000000,0.000000,68.516280,67.900000,11,5,1
2014-11-01 01:10:00-04:00,42.989980,86.510020,0.489018,1.250000,0.000000,68.658485,67.900000,11,5,1
2014-11-01 01:20:00-04:00,42.900000,86.600000,0.605010,1.124740,0.000000,68.700000,67.900000,11,5,1


In [2]:
#Move at (the y) to the end of the last column of the dataframe
at = train['at'];
train.drop(labels=['at'], axis=1,inplace = True)
train.insert(len(train.columns), 'at', at)

at = test['at'];
test.drop(labels=['at'], axis=1,inplace = True)
test.insert(len(test.columns), 'at', at)
test

/home/zhiang/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/zhiang/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,oa,rh,sr,ws,dp,st,month,weekday,hour,at
time,,,,,,,,,,
2015-11-01 00:10:00-04:00,55.811670,57.576660,1.297827,1.322937,0.0,69.288820,11,6,0,68.292000
2015-11-01 00:20:00-04:00,56.600000,54.911670,0.565834,3.864502,0.0,69.188680,11,6,0,68.200000
2015-11-01 00:30:00-04:00,56.611477,54.788520,0.654262,7.161926,0.0,69.088810,11,6,0,68.200000
2015-11-01 00:40:00-04:00,58.188522,54.500000,0.586068,8.391342,0.0,69.011314,11,6,0,68.208000
2015-11-01 00:50:00-04:00,60.210003,48.429990,0.995001,3.937762,0.0,69.100000,11,6,0,68.200000
2015-11-01 01:00:00-04:00,61.089996,44.900000,0.920994,4.249959,0.0,69.111040,11,6,1,68.200000
2015-11-01 01:10:00-04:00,60.576656,43.900000,0.465836,2.020896,0.0,69.222664,11,6,1,68.200000
2015-11-01 01:20:00-04:00,59.188330,44.123337,0.890000,5.697937,0.0,69.388670,11,6,1,68.200000
2015-11-01 01:30:00-04:00,58.076660,47.135010,0.840000,4.739687,0.0,69.288690,11,6,1,68.200000


In [3]:
#The error metric
def chouErrorMetric(pred, real):
    """
    Reference:
    Jui Sheng Chou and Dac Khuong Bui. Modeling heating and cooling
    loads by artificial intelligence for energy-efficient building design.
    Energy and Buildings, 82:437–446, 2014.
    
    pred: numpy array of the predicted value
    real: numpy array of the real value
    """
    n = pred.shape[0];
    rmse = (
            (np.sum((pred-real)**2.0)/n)**0.5
            );
    
    mae = (
            np.sum(np.absolute((pred-real)))/n
            );
    
    r = (
         (np.sum(np.multiply(real,pred))*n 
        - np.sum(real)*np.sum(pred))/
        (np.sqrt(n*np.sum(real**2)-(np.sum(real)**2))*
         np.sqrt(n*np.sum(pred**2)-(np.sum(pred)**2)))
            );
    
    r_normal = (1-r)/2.0 #make r value between 0 and 1, 0 is the best
    errorList = [rmse, mae, r_normal];
    print ("error list",errorList);
    avgError = sum(errorList)/3.0;
    return avgError,errorList;
    
    
    
    
    
    
    
    
    

In [4]:
#min-max normalization
def normalize(data):
    """
    min max normalization
    data: np array, n*(m+1), n is sample number,
          m is feature number, plus 1 for the true value
    return: (np array 1*(m+1) for the max-min for each column,
             np array 1*(m+1) for the minimum value for each colmn)
    """
    colNum = data.shape[1];
    division = [];
    minVList = [];
    for i in range(colNum):
        col = data[:, i];
        maxV = (np.max(col));
        minV = (np.min(col));
        division.append(maxV-minV);
        minVList.append(minV);
        data[:,i] = data[:,i] - minV;

    division = np.array(division);
    minVList = np.array(minVList);
    return (np.true_divide(data, division[None,:]), division, minVList);
    
    
    

In [5]:
#Lazy learning

In [6]:

#lazy learning with non-weighted average kernel
class lazyLearning():
    
    def __init__(self, train, k):

        """
        perform lazy learning with linear average kernel and 
        eculidean distance.
        inputs: 
        train: numpy array, n*(m+1),  
               each row is a sample (total n samples),
               each col is a feature (total m features), 
               the last col is the real y
        k: k-nn in lazy learning
        """
        self.trainFeat = train[:,0:-1];
        self.trainY = train[:,-1];
        self.k = k;
    
    def predict(self,test):
        """
        test: numpy array 1*m, m is the number of feature
        return the predicted value
        """
        eclideanDist = np.linalg.norm(self.trainFeat - test, axis = 1);
        eclideanDistSort = eclideanDist.argsort(axis = 0);
        eclideanDistSort = eclideanDistSort[0:self.k];
        knneighbours = ([self.trainY[index] for 
                         index in eclideanDistSort]);
        yt = np.mean(knneighbours);
        return yt;

#cross validation

def getCrossValidationData(trainArray, fold):
    """
    get the data list for cross validation
    trainArray: raw train data, numpy array n*(m+1)
                where n is sample size, m is feature
                size, plus 1 for the true result
    fold: fold value
    return tuple ([trainNpArray_1, trainNpArray_2,...]
                 ,[testNpArray_1, testNpArray_2,...])
    """
    trainLength = trainArray.shape[0];
    numEachFold = int(trainLength/10);
    import random;
    np.random.shuffle(trainArray);#shuffle rows
    cvTrainList = [];
    cvTestList = [];
    for foldK in range(fold):
        foldStart = int(foldK*(trainLength)/fold);
        foldEnd = int(foldStart + trainLength/fold);
        if foldEnd >= trainLength:
            foldEnd = trainLength - 1;
        cvTest = trainArray[foldStart:foldEnd,:];
        cvTrain = np.delete(trainArray
                            ,range(foldStart, foldEnd, 1)
                            ,axis = 0);
        cvTrainList.append(cvTrain);
        cvTestList.append(cvTest);
    return (cvTrainList, cvTestList);


def crossValidateForChioces(choiceList, trainArray, mlMethod,
                            fold = 10):
    """
    Perform cross validation to select model.
    choiceList: 1D list containing different numeric choice
    trainArray: np array n*(m+1) for the training
    mlMethod: ml class
    fold: cross validation fold
    return: cross validation error using the chouErrorMetric
            over the choice list
    """
    cvTrainList, cvTestList = (
        getCrossValidationData(trainArray, fold));
    print ('Performing cross validation to find the best k...');
    cvErrorOverK = [];
    for k in choiceList:
        print ('Testing k=', k, ' ...')
        error_k = [];
        for fold_i in range(len(cvTrainList)):
            print ('Testing k=',k,' for fold=',fold_i,' ...')
            pred_fold_i = [];
            cvTrain_i = cvTrainList[fold_i];
            cvTest_i = cvTestList[fold_i];
            model = mlMethod(cvTrain_i,k);
            for testSample_i in range(len(cvTest_i)):
                pred_fold_i.append(model
                          .predict(cvTest_i[testSample_i,0:-1]));
            pred_fold_i = np.array(pred_fold_i);
            error_fold_i = chouErrorMetric(pred_fold_i,cvTest_i[:,-1]);
            print ('Error metric for k=',k,' fold=', fold_i,
              ' is ', error_fold_i);
            error_k.append(error_fold_i[0]);
        meanErrorFold_i = np.mean(np.array(error_k));
        print ('Cross validated error metric for k=',k,
          ' is ',meanErrorFold_i);
        cvErrorOverK.append(np.mean(np.array(error_k)));
    return cvErrorOverK


In [7]:
#Perform cross validation to choose K for LL-Avg
trainArray = train.as_matrix();#non-normalized data
trainArrayNorm, normDivision, normMinList = (
    normalize(train.as_matrix()));#normalized data
llKChoiceList = range(5,100,5);
trainArrayNorm
cvErrorOverK = crossValidateForChioces(
                llKChoiceList
                ,trainArrayNorm
                ,lazyLearning
                ,fold = 10
                );


Performing cross validation to find the best k...
Testing k= 5  ...
Testing k= 5  for fold= 0  ...


KeyboardInterrupt: 

In [8]:
#plot error metric over K for LL-Avg
print (cvErrorOverK)

plt.plot(llKChoiceList,cvErrorOverK,
            color="b",
            label="Lazy Learning with Uniform Average Kernel");
plt.xlabel(
            "Choice of K",
            fontsize = 14
            );
plt.ylabel(
            "Cross Validation Error (Error Metric)",
            fontsize = 14
            );
plt.legend();
plt.show();

NameError: name 'cvErrorOverK' is not defined

In [ ]:
#LL-Avg: Test over the next year's data
testArray = test.as_matrix();
optK = llKChoiceList[np.argmin(np.array(cvErrorOverK))];

modelData = train.as_matrix();
modelDataLen = modelData.shape[0];
allData_test = np.append(modelData, testArray, 0); #model data + test array
#normalize both model data and test data
#to the same scale
allData_test,normDivision_test,normMinList_test = (
                                                normalize (allData_test));
#modelData: the dataset pool for LL
#testArray: the different dataset for the tesing
modelData = allData_test[0:modelDataLen,:];
testArray = np.delete(allData_test
                     ,range(modelDataLen)
                     ,axis = 0);

llTest = lazyLearning(modelData, optK);

testRest_norm = [];
for sample_i in range(len(testArray)):
    testRest_norm.append(llTest
                    .predict(testArray[sample_i,0:-1]));
testRest_norm = np.array(testRest_norm);
(testError_norm, detail) = chouErrorMetric(testRest_norm, testArray[:,-1]);
print ("Test error metric over the test data for LL-Avg is ",testError_norm,".");

#plot test data, pred vs. true (normalized)
plt.plot(testRest_norm
         ,testArray[:,-1]
         ,'.'
         ,color="b"
         ,label="Lazy Learning with Uniform Average Kernel");

plt.xlabel(
            "Predicted Value for the Test Data (Min-max Normalized)",
            fontsize = 12
            );
plt.ylabel(
            "True Value for the Test Data (Min-max Normalized)",
            fontsize = 12
            );
plt.legend();
plt.show();
#plot test data, random 1000 non-consective pred and true over time index (unnormalized)
testRest_unNorm = testRest_norm*normDivision_test[-1]+normMinList_test[-1];
testRest_unNorm = testRest_unNorm.reshape((1,len(testRest_unNorm)));
trueRest_unNorm = testArray[:,-1]*normDivision_test[-1]+normMinList_test[-1];
trueRest_unNorm = trueRest_unNorm.reshape((1,len(trueRest_unNorm)));
testNtrueRest_unNorm = np.append(testRest_unNorm, trueRest_unNorm, axis = 0);

plt.plot(testNtrueRest_unNorm[0,0:2000]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,0:2000]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-01-01 to 2015-01-14",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("Lazy Learning with Uniform Average Kernel");
plt.legend();
plt.show();

plt.plot(testNtrueRest_unNorm[0,11233:13233]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,11233:13233]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-03-20 to 2015-04-02",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("Lazy Learning with Uniform Average Kernel");
plt.legend();
plt.show();

plt.plot(testNtrueRest_unNorm[0,21745:23745]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,21745:23745]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-06-01 to 2015-06-14",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("Lazy Learning with Uniform Average Kernel");
plt.legend();
plt.show();
    
plt.plot(testNtrueRest_unNorm[0,42049:44049]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,42049:44049]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-10-20 to 2015-11-02",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("Lazy Learning with Uniform Average Kernel");
plt.legend(loc='lower left');
plt.show();
    
    
    
    

In [ ]:

def makeLaggedData(lagOrder, rawData):
    """
    add lagged feature to the original dataset
    :lagOrder type
        1D list of float
    :lagOrder para,
        lagOrder corresponding to each column of the raw data
        e.g. [2,1,1] means for the first column (the first feature) in
            the rawData, add 1&2 time steps lagged data as the new features;
            for the second column (the original second feature) in the
            rawData, add 1 step lagged data as the new feature. 
    :rawData: numpy array, n*(m+1)
    return: new dataset with lagged features
    """
    colNum = rawData.shape[1];
    rowNum = rawData.shape[0];
    autoRegNum = len(lagOrder);
    returnedData = [];
    maxRegOrder = max(lagOrder);
    for i in range(colNum):
        
        if i < autoRegNum:
            autoRegOrder = lagOrder[i];
            for j in range(0,autoRegOrder):
                appendToHead = np.zeros(j+1);
                returnedData.append(
                        np.append(appendToHead, 
                            rawData[0:rowNum-j-1,i]));
        returnedData.append(rawData[:,i]);
    
    returnedData = np.array(returnedData).transpose();
    returnedData = returnedData[maxRegOrder:,:];
    return returnedData;

#lazy learning with linear regression as the local model
class lazyLearning_localLinear():
    
    def __init__(self, train, k):

        """
        perform lazy learning with linear average kernel and 
        eculidean distance.
        inputs: 
        train: numpy array, n*(m+1),  
               each row is a sample (total n samples),
               each col is a feature (total m features), 
               the last col is the real y
        k: k-nn in lazy learning
        """
        self.trainFeat = train[:,0:-1];
        self.trainY = train[:,-1];
        self.k = k;
    
    def predict(self,test):
        """
        test: numpy array 1*m, m is the number of feature
        return the predicted value (scalar)
        """
        eclideanDist = np.linalg.norm(self.trainFeat - test, axis = 1);
        eclideanDistSort = eclideanDist.argsort(axis = 0);
        eclideanDistSort = eclideanDistSort[0:self.k];
        knneighboursY = np.array([self.trainY[index] for 
                         index in eclideanDistSort]).reshape(self.k,1);
        m = self.trainFeat.shape[1];
        knneighboursX = self.trainFeat[eclideanDistSort[0],:].reshape(1,m);
        for index in eclideanDistSort[1:]:
            knneighboursX = np.append(
                knneighboursX, self.trainFeat[index,:].reshape(1,m),0);
        from sklearn import linear_model
        regr = linear_model.LinearRegression(n_jobs = -1);
        regr.fit(knneighboursX, knneighboursY)
        test = test.reshape(1,m);
        return regr.predict(test)[0,0];
    


In [ ]:
#Cross validation choose k for lazy learning with linear regression
#Perform cross validation
trainArray = train.as_matrix();#non-normalized data
trainArrayNorm, normDivision, normMinList = (
    normalize(train.as_matrix()));#normalized data

llKChoiceList = range(100,500,100);
cvErrorOverK = crossValidateForChioces(
                llKChoiceList
                ,trainArrayNorm
                ,lazyLearning_localLinear
                ,fold = 10
                );

In [ ]:
cvErrorOverK

In [ ]:
#ll with uniform average kernel, but with lagged features
#Perform cross validation
trainArray = train.as_matrix();#non-normalized data
trainArrayNorm, normDivision, normMinList = (
    normalize(train.as_matrix()));#normalized data
lagOrder = [2,0,2,0,0,0,0,0,0,2];
trainArrayNormLagged = makeLaggedData(lagOrder, trainArrayNorm)
llKChoiceList = range(5,50,5);
cvErrorOverK = crossValidateForChioces(
                llKChoiceList
                ,trainArrayNormLagged
                ,lazyLearning
                ,fold = 10
                );

In [ ]:
cvErrorOverK_0 = cvErrorOverK

In [ ]:
#Test over the test dataset for LL-LR
testArray = test.as_matrix();
optK = 100;

modelData = train.as_matrix();
modelDataLen = modelData.shape[0];
allData_test = np.append(modelData, testArray, 0); #model data + test array
allData_test,normDivision_test,normMinList_test = (
                                                normalize (allData_test));
modelData = allData_test[0:modelDataLen,:];
testArray = np.delete(allData_test
                     ,range(modelDataLen)
                     ,axis = 0);

llTest = lazyLearning_localLinear(modelData, optK);

testRest_norm = [];
for sample_i in range(len(testArray)):
    testRest_norm.append(llTest
                    .predict(testArray[sample_i,0:-1]));
testRest_norm = np.array(testRest_norm);

(testError_norm, detail) = chouErrorMetric(testRest_norm, testArray[:,-1]);
print ("Test error metric over the test data is ",testError_norm,".");
print (detail)

#plot test data, pred vs. true (normalized)
plt.plot(testRest_norm
         ,testArray[:,-1]
         ,'.'
         ,color="b"
         ,label="Lazy Learning with Linear Regression Kernel");

plt.xlabel(
            "Predicted Value for the Test Data (Min-max Normalized)",
            fontsize = 10
            );
plt.ylabel(
            "True Value for the Test Data (Min-max Normalized)",
            fontsize = 10
            );
plt.axis([0,1,0,1])
plt.legend();
plt.show();
#plot test data, random 1000 non-consective pred and true over time index (unnormalized)
testRest_unNorm = testRest_norm*normDivision_test[-1]+normMinList_test[-1];
testRest_unNorm = testRest_unNorm.reshape((1,len(testRest_unNorm)));
trueRest_unNorm = testArray[:,-1]*normDivision_test[-1]+normMinList_test[-1];
trueRest_unNorm = trueRest_unNorm.reshape((1,len(trueRest_unNorm)));
testNtrueRest_unNorm = np.append(testRest_unNorm, trueRest_unNorm, axis = 0);

plt.plot(testNtrueRest_unNorm[0,0:2000]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,0:2000]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-01-01 to 2015-01-14",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("Lazy Learning with Linear Regression Kernel");
plt.legend();
plt.show();

plt.plot(testNtrueRest_unNorm[0,11233:13233]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,11233:13233]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-03-20 to 2015-04-02",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("Lazy Learning with Linear Regression Kernel");
plt.legend();
plt.show();

plt.plot(testNtrueRest_unNorm[0,21745:23745]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,21745:23745]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-06-01 to 2015-06-14",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("Lazy Learning with Linear Regression Kernel");
plt.legend();
plt.show();
    
plt.plot(testNtrueRest_unNorm[0,42049:44049]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,42049:44049]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-10-20 to 2015-11-02",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("Lazy Learning with Linear Regression Kernel");
plt.legend(loc='lower left');
plt.show();

In [ ]:
#Test over the test dataset for LL-Avg-Lag
cvErrorOverK = [0.006261511172007253,
                0.0078319748087286763,
                0.0092544725368474655,
                0.010491375591088101,
                0.011582034373530298,
                0.012577092631859799,
                0.013481123234175307,
                0.014271828395324013,
                0.014997619668965751]#copied from the previous computation

testArray = test.as_matrix();
optK = llKChoiceList[np.argmin(np.array(cvErrorOverK))];

modelData = train.as_matrix();
modelDataLen = modelData.shape[0];
allData_test = np.append(modelData, testArray, 0); #model data + test array
allData_test,normDivision_test,normMinList_test = (
                                                normalize (allData_test));
lagOrder = [2,0,2,0,0,0,0,0,0,2];
allData_test = makeLaggedData(lagOrder, allData_test);                                                                          #to the same scale
modelData = allData_test[0:modelDataLen,:];
testArray = np.delete(allData_test
                     ,range(modelDataLen)
                     ,axis = 0);

llTest = lazyLearning(modelData, optK);

testRest_norm = [];
#perform test.
#because lagged indoor air temperature (T_k-1, T_k-2, 
#k is the time step that we want to predict the T)
#is an input to the model, so the prediction
#is performed in an iterative way, that is:
#test data is divided into groups and each group
#contains the consectively 1000 sampes.
#For every 1000 test samples, the first sample's
#T_k-1 and T_k-2 is the real measured data; the
#second sample's T_k-1 is the predicted value from
#the last sample; the third sample's T_k-1 and 
#T_k-2 are the predicted values from the last two
#samples, so on and so forth. 
lagOrderT = lagOrder[-1];
test_sample_i = testArray[0,0:-1];
for sample_i in range(len(testArray)):
    pred_i = llTest.predict(test_sample_i);
    testRest_norm.append(pred_i);
    if sample_i + 1 < len(testArray):
        test_sample_i_new = testArray[sample_i+1,0:-1];
        sampleFeatM = len(test_sample_i_new);
        test_sample_i_new[sampleFeatM-lagOrderT] = pred_i;
        for lagOrderT_i in range(1,lagOrderT):
            test_sample_i_new[sampleFeatM - lagOrderT 
                              + lagOrderT_i] = test_sample_i[sampleFeatM 
                                                             - lagOrderT + lagOrderT_i - 1]
        test_sample_i = test_sample_i_new;
        
testRest_norm = np.array(testRest_norm);

In [ ]:
#continue...
(testError_norm, detail) = chouErrorMetric(testRest_norm, testArray[:,-1]);
print ("Test error metric over the test data is ",testError_norm,".");
print (detail)

#plot test data, pred vs. true (normalized)
plt.plot(testRest_norm
         ,testArray[:,-1]
         ,'.'
         ,color="b"
         ,label="LL with Average Kernel - Lagged Features");
plt.axis([0,1,0,1]);

plt.xlabel(
            "Predicted Value for the Test Data (Min-max Normalized)",
            fontsize = 10
            );
plt.ylabel(
            "True Value for the Test Data (Min-max Normalized)",
            fontsize = 10
            );
plt.legend();
plt.show();
#plot test data, samples from each season
testRest_unNorm = testRest_norm*normDivision_test[-1]+normMinList_test[-1];
testRest_unNorm = testRest_unNorm.reshape((1,len(testRest_unNorm)));
trueRest_unNorm = testArray[:,-1]*normDivision_test[-1]+normMinList_test[-1];
trueRest_unNorm = trueRest_unNorm.reshape((1,len(trueRest_unNorm)));
testNtrueRest_unNorm = np.append(testRest_unNorm, trueRest_unNorm, axis = 0);

plt.plot(testNtrueRest_unNorm[0,0:2000]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,0:2000]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-01-01 to 2015-01-14",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("LL with Average Kernel - Lagged Features");
plt.legend();
plt.show();

plt.plot(testNtrueRest_unNorm[0,11000:13000]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,11000:13000]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-03-19 to 2015-04-01",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("LL with Average Kernel - Lagged Features");
plt.legend(fontsize = 12);
plt.show();

plt.plot(testNtrueRest_unNorm[0,21800:23800]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,21800:23800]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-06-02 to 2015-06-15",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("LL with Average Kernel - Lagged Features");
plt.legend();
plt.show();
    
plt.plot(testNtrueRest_unNorm[0,42000:44000]
         ,color="b"
         ,label="Predicted Value");
plt.plot(testNtrueRest_unNorm[1,42000:44000]
         ,color='r'
         ,label='True Value');
plt.axis([0,2000,50,80]);
plt.xlabel(
            "2015-10-20 to 2015-11-02",
            fontsize = 14
            );
plt.ylabel(
            "Indoor Air Temperature (F)",
            fontsize = 14
            );
plt.title("LL with Average Kernel - Lagged Features");
plt.legend(loc='lower left');
plt.show();